In [1]:
import os

import matplotlib.animation as animation
import matplotlib.pyplot as plt
import numpy as np
import torch
from IPython.display import HTML

In [2]:
os.chdir('/gpfs/space/home/joonas97/')
os.getcwd()

'/gpfs/space/home/joonas97'

In [3]:
cams = np.load("./KITSCAM/3d/results/scorecam_3d_artifacts/2022-12-12/15-12-48/cams.npy", allow_pickle=True)

In [4]:
cams.shape

(60,)

In [5]:

images = np.load("./KITSCAM/3d/results/scorecam_3d_artifacts/2022-12-12/15-12-48/inputs.npy", allow_pickle=True)
labels = np.load("./KITSCAM/3d/results/scorecam_3d_artifacts/2022-12-12/15-12-48/labels.npy", allow_pickle=True)

In [6]:
cams_block4 = np.load("./KITSCAM/3d/results/scorecam_3d_artifacts/2022-12-13/11-59-16/cams.npy", allow_pickle=True)

In [40]:
cams_block3 = np.load("./KITSCAM/3d/results/scorecam_3d_artifacts/2022-12-13/13-10-05/cams.npy", allow_pickle=True)
cams_block2 = np.load("./KITSCAM/3d/results/scorecam_3d_artifacts/2022-12-13/13-20-34/cams.npy", allow_pickle=True)

In [86]:
cams_block1 = np.load("./KITSCAM/3d/results/scorecam_3d_artifacts/2022-12-13/13-46-25/cams.npy", allow_pickle=True)

In [87]:
cams_block0 = np.load("./KITSCAM/3d/results/scorecam_3d_artifacts/2022-12-13/13-49-19/cams.npy", allow_pickle=True)

In [92]:
def get_relevant_metadata(image):
    path = image.meta["filename_or_obj"][0]
    #print(path)
    splitted_path = path.split("/")
    #slice_indxs = splitted_path[-1]
    #first, second = slice_indxs.strip(".npy").split("_")
    #print(splitted_path)
    patient_id = splitted_path[-1].strip(".npy")
    return patient_id  #int(first), int(second)


def get_animation(volume):
    fig = plt.figure()

    ims = []
    for image in range(0, volume.shape[0]):
        im = plt.imshow(volume[image, :, :],
                        animated=True, cmap=plt.cm.bone)

        plt.axis("off")
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=100, blit=False,
                                    repeat_delay=1000)

    plt.close()
    return ani


def get_full_animation(volume, cams, cams_meta):
    fig = plt.figure()

    nr_of_rows = 1
    nr_of_cols = len(cams) + 1
    if len(cams) > 3:
        nr_of_rows = 2
        nr_of_cols = nr_of_cols // 2 + 1

    fig, (ax, ax2) = plt.subplots(nr_of_rows, nr_of_cols, figsize=(14, 7))
    fig.delaxes(ax2[3])
    ims = []
    slices = []
    for image in range(0, volume.shape[0]):

        slices = []
        im1 = ax[0].imshow(volume[image, :, :],
                           animated=True, cmap=plt.cm.bone)
        ax[0].set_title("Input image")
        #plt.axis("off")
        slices.append(im1)

        for i, cam in enumerate(cams):
            current_ax = (ax if i + 1 < nr_of_cols else ax2)
            current_index = (i + 1 if i + 1 < nr_of_cols else i + 1 - nr_of_cols)

            im = current_ax[current_index].imshow(cam[image, :, :],
                                                  animated=True, cmap=plt.cm.bone)
            current_ax[current_index].set_title(cams_meta[i])
            current_ax[current_index].axis('off')
            slices.append(im)

        ims.append(slices)

    #plt.axis("off")  
    ani = animation.ArtistAnimation(fig, ims, interval=100, blit=False,
                                    repeat_delay=1000)
    plt.close()
    return ani

In [94]:
import matplotlib

matplotlib.rcParams['animation.embed_limit'] = 2 ** 128

In [ ]:
animations = []
for i in range(30):
    patient_id = get_relevant_metadata(images[i])
    #path = '/gpfs/space/home/joonas97/data/kits21/kits21/data/' + patient_id + '/aggregated_MAJ_seg.nii.gz'
    #seg = nib.load(path).get_fdata()[start:stop]
    print("-----" * 10)
    print("Animation nr: ", i)
    inp = torch.squeeze(images[i].as_tensor())
    cam1 = torch.squeeze(cams[i].as_tensor())
    cam2 = torch.squeeze(cams_block4[i].as_tensor())
    cam3 = torch.squeeze(cams_block3[i].as_tensor())
    cam4 = torch.squeeze(cams_block2[i].as_tensor())
    cam5 = torch.squeeze(cams_block1[i].as_tensor())
    cam6 = torch.squeeze(cams_block0[i].as_tensor())
    cams_meta = ["320 6x16x16 block6",
                 "112 12x32x32 block4",
                 "80 12x32x32 block3",
                 "40 40x25x64x64 block2",
                 "24 50x128x128 block1",
                 "16 100x256x256 block0"]
    ani = get_full_animation(inp, [cam1, cam2, cam3, cam4, cam5, cam6], cams_meta)
    animations.append(ani)
    display(HTML(ani.to_jshtml()))

--------------------------------------------------
Animation nr:  0
2022-12-13 14:28:05,079 - Animation.save using <class 'matplotlib.animation.HTMLWriter'>


In [ ]:
for i in range(7, 15):
    patient_id = get_relevant_metadata(images[i])
    #path = '/gpfs/space/home/joonas97/data/kits21/kits21/data/' + patient_id + '/aggregated_MAJ_seg.nii.gz'
    #seg = nib.load(path).get_fdata()[start:stop]

    inp = torch.squeeze(images[i].as_tensor())
    cam1 = torch.squeeze(cams[i].as_tensor())
    cam2 = torch.squeeze(cams_block4[i].as_tensor())

    seg_ani = get_double_animation(inp, )
    display(HTML(seg_ani.to_jshtml()))

In [ ]:
for i in range(4):
    patient_id = get_relevant_metadata(images[i])
    #path = '/gpfs/space/home/joonas97/data/kits21/kits21/data/' + patient_id + '/aggregated_MAJ_seg.nii.gz'
    #seg = nib.load(path).get_fdata()[start:stop]

    inp = torch.squeeze(images[i].as_tensor())
    cam1 = torch.squeeze(cams[i].as_tensor())
    cam2 = torch.squeeze(cams_block4[i].as_tensor())

    seg_ani = get_double_animation(inp, cam1, cam2)
    display(HTML(seg_ani.to_jshtml()))


In [ ]:
for i in range(5, 10):
    patient_id = get_relevant_metadata(images[i])
    #path = '/gpfs/space/home/joonas97/data/kits21/kits21/data/' + patient_id + '/aggregated_MAJ_seg.nii.gz'
    #seg = nib.load(path).get_fdata()[start:stop]

    inp = torch.squeeze(images[i].as_tensor())
    current_cam = torch.squeeze(cams[i].as_tensor())
    seg_ani = get_double_animation(inp, current_cam)
    display(HTML(seg_ani.to_jshtml()))